In [54]:
import numpy as np
import matplotlib.pyplot as plt

In [55]:
class Body:
    def __init__(self, position, velocity, mass):
        self.position = position
        self.velocity = velocity
        self.mass = mass
    
    def calculate_force_on_body(self, body):
        if(body == self):
            return np.array([0,0,0])
        distance = self.calculate_distance_to_body(body)
        return (self.position - body.position) * (self.mass * body.mass) / distance**3
    
    def calculate_potential_of_body(self, body):
        if(body == self):
            return 0.0
        distance = self.calculate_distance_to_body(body)
        return -(self.mass * body.mass) / distance
    
    def calculate_distance_to_body(self, body):
        position_vector = body.position - self.position
        return np.sqrt(np.sum(position_vector**2))
        
class System:
    def __init__(self):
        self.bodies = []
        
    def add_body(self, body):
        self.bodies.append(body)
        
    def reset(self):
        self.bodies = []
    
    def calculate_total_force_on_body(self, body):
        force = np.zeros(3)
        for system_body in self.bodies:
            force += system_body.calculate_force_on_body(body)
        return force
    
    def calculate_average_distance_between_bodies(self):
        distance = 0.0
        number_of_bodies = len(self.bodies)
        for i in range(number_of_bodies):
            for j in range(i+1, number_of_bodies):
                distance += self.bodies[i].calculate_distance_to_body(self.bodies[j])
        return distance / (number_of_bodies * (number_of_bodies + 1) / 2)
    
    def calculate_total_energy(self):
        kinetic_energy = 0.0
        potential_energy= 0.0
        number_of_bodies = len(self.bodies)
        for i in range(number_of_bodies):
            kinetic_energy += 0.5 * self.bodies[i].mass * (np.linalg.norm(self.bodies[i].velocity))**2
            for j in range(i+1, number_of_bodies):
                potential_energy += self.bodies[i].calculate_potential_of_body(self.bodies[j])
        return kinetic_energy + potential_energy
        
class Simulation:
    def __init__(self, system = System()):
        self.time = 0
        self.delta_t = 0.001
        self.simulated_system = system
        self.animation = Animation()
        
    def propagate(self):
        initial_forces = np.zeros((len(self.simulated_system.bodies), 3))
        new_forces = np.zeros((len(self.simulated_system.bodies), 3))
        
        # calculate old forces
        for i, body in enumerate(self.simulated_system.bodies):
            initial_forces[i] = self.simulated_system.calculate_total_force_on_body(body)
        
        # propagete position
        for i, body in enumerate(self.simulated_system.bodies):
            body.position = body.position + body.velocity * self.delta_t + 0.5 * initial_forces[i] * (self.delta_t)**2 / body.mass
        
        # recalculate forces
        for i, body in enumerate(self.simulated_system.bodies):
            new_forces[i] = self.simulated_system.calculate_total_force_on_body(body)
            
        for i, body in enumerate(self.simulated_system.bodies):
            body.velocity = body.velocity + 0.5 * (initial_forces[i] + new_forces[i]) * (self.delta_t) / body.mass
    
    # simulate for "simulation time" seconds
    def simulate(self, simulation_time):
        start_time = self.time
        if self.simulated_system.calculate_total_energy() > 0:
            print("WARNING: system could be unstable")
        while(self.time < start_time + simulation_time):
            self.propagate()
            self.animation.add_frame(Frame(self.simulated_system.bodies))
            self.time += self.delta_t
            
    def reset(self):
        self.time = 0
        self.animation = Animation()
            
class Animation:
    def __init__(self):
        self.frames = []
        
    def add_frame(self, frame):
        self.frames.append(frame)
        
    def run(self):
        # need to define figure/ axis, animate function i.e. one called each time to render frames, len(y) = len frames
        # interval needs to be figured out
        ani = animation.FuncAnimation(fig, animate, len(y), interval=dt*1000, blit=True)
        plt.show()
            
class Frame:
    def __init__(self, bodies):
        self.positions = [body.position for body in bodies]
    
    def get_2D_positions(self):
        return [position[[0,2]] for position in self.positions]

In [56]:
system = System()
simulation = Simulation(system)

system.reset()
body1 = Body(np.array([0,0,-1]), np.array([0.5,0,0.0]), 1)
body2 = Body(np.array([0,0,1]), np.zeros(3), 1)
body3 = Body(np.array([0,1,0]), np.zeros(3), 1)
system.add_body(body1)
system.add_body(body2)
system.add_body(body3)

2
3
